In [1]:
import re
from collections import Counter
from pprint import pprint

# load input from "day19_input.txt"

In [2]:
def parse_input(fn_input):
    scans = {}
    
    pattern = re.compile("--- scanner (\d+) ---")
    
    with open(fn_input, "r") as f:
        for line in f.readlines():
            if line.startswith("---"):
                # start a new scanner
                i = int(pattern.findall(line)[0])
                assert i not in scans
                scans[i] = []
            elif len(line.strip()) == 0:
                continue
            else:
                pos = [int(a) for a in line.strip().split(",")]
                scans[i].append(pos)
    return scans
            

In [3]:
def rotate_point_90(p, dim, n=1):
    if n== 0:
        return p
    
    if dim==0:
        p2 = (p[0], -p[2], p[1])
    elif dim==1:
        p2 = (-p[2], p[1], p[0])
    elif dim==2:
        p2 = (-p[1], p[0], p[2])
    else:
        raise NotImplementedError
    
    if n==1:
        return p2
    else:
        assert n>1
        return rotate_point_90(p2, dim, n-1)

def rotate_beacons(beacon_list, dim, n):
    return [rotate_point_90(p, dim, n) for p in beacon_list]

In [4]:
# test rotation
a = [1, 2, 3]
for n in range(5):
    print(rotate_point_90(a, 1, n))
# note: the last one must be same as first one

[1, 2, 3]
(-3, 2, 1)
(-1, 2, -3)
(3, 2, -1)
(1, 2, 3)


In [5]:
def rotate_all(bl):
    rotated = []
    for nx in range(4):
        t1 = rotate_beacons(bl, 0, nx)
        for ny in range(4):
            t2 = rotate_beacons(t1, 1, ny)
            for nz in range(4):
                t3 = rotate_beacons(t2, 2, nz)
                rotated.append((t3, (nx, ny, nz)))
    return rotated

In [6]:
def match_x(bl1, bl2):
    distances = [a[0]-b[0] for a in bl1 for b in bl2]
    counts = Counter(distances)
    possible = []
    for distance, count in counts.most_common():
        if count < 12:
            break
        pairs = [(i1, i2) for i1, a in enumerate(bl1) for i2, b in enumerate(bl2) if (a[0]-b[0]) == distance]
        possible.append((pairs, distance))
    # print("x found {}".format(len(possible)))
    return possible


def filter_yz(possible, bl1, bl2):
    checked = []
    for pairs, offset_x in possible:
        offsets = [offset_x]
        for dim in [1, 2]:
            distance = [bl1[i1][dim]-bl2[i2][dim] for (i1, i2) in pairs]
            d, n = Counter(distance).most_common(1)[0]
            if n < 12:
                break # fail
            offsets.append(d)
        else:
            # now success
            checked.append((tuple(pairs), tuple(offsets)))
    return checked


def match_beacons(bl1, bl2):
    return filter_yz(match_x(bl1, bl2), bl1, bl2)

In [7]:
def get_chains(ks, n_key=31):
    chains = {0:None}
    for i in range(n_key):
        for k1, k2 in ks:
            # print(k1, k2)
            if k2 not in chains:
                if k1 == 0:
                    chains[k2] = [k1, k2]
                else:
                    if k1 in chains:
                        chains[k2] = chains[k1]+[k2]
                    
    return chains

In [8]:
def transfer_scanner(bl, rotates, offsets):
    t1 = bl
    for i_dim in range(3):
        t1 = rotate_beacons(t1, i_dim, rotates[i_dim])
    t1 = [(x+offsets[0], y+offsets[1], z+offsets[2]) for x,y,z in t1]
    return t1

def transfer_scanner_final(bl, convert_chain, between_scanners):
    t = bl
    for i_step in list(range(len(convert_chain)-1, 0, -1)):
        s_from = convert_chain[i_step-1]
        s_to = convert_chain[i_step]
        rotates, offsets = between_scanners[(s_from, s_to)]
        t = transfer_scanner(t, rotates, offsets)
    return t

In [9]:
def manhattan_distance(p1, p2):
    return abs(p1[0]-p2[0]) + abs(p1[1]-p2[1]) + abs(p1[2]-p2[2])

# program start

In [10]:
scans = parse_input("day19_input.txt")
# print(scans)

scanner_index = list(range(len(scans)))
for k, v in scans.items():
    print("{}: {}".format(k, len(v)))
    
# NOTE: the beacons each scanner found are from 25 ~ 27

0: 26
1: 25
2: 26
3: 26
4: 26
5: 25
6: 27
7: 27
8: 26
9: 26
10: 26
11: 26
12: 26
13: 26
14: 25
15: 26
16: 26
17: 27
18: 26
19: 26
20: 26
21: 26
22: 27
23: 25
24: 25
25: 26
26: 27
27: 26
28: 27
29: 26
30: 26


In [11]:
between_scanners = {}
for k1, v1 in scans.items():
    for k2, v2 in scans.items():
        if k1 == k2:
            continue
            
        possible_all = []
        for i_rota, (bl2, rotates) in enumerate(rotate_all(v2)):
            # print("new rotate:", k1, k2, i_rota, rotates, bl2)
            possible = match_beacons(v1, bl2)
            assert len(possible) <=1
            if len(possible) == 0:
                continue
            # print(possible)
            pairs, offsets = possible[0]
            possible_all.append((rotates, offsets))
                # print("matched: {} vs {}, rotation:{}".format(k1, k2, rotates))
        
        if len(possible_all)> 0:
            # some diff by rotations. just pick 1st one.
            between_scanners[(k1, k2)] = possible_all[0]

pprint(between_scanners)

# NOTE: what is inside between scanner:
# (0, 12): ((0, 1, 0), (1117, 115, 56))
# it means convert scanner 12 to scanner 0 
# (yes i made the wrong order, but too lazy to change it)
# the rotation: (0, 1, 0) means 90degree on y-axis
# the offset: (1117, 115, 56) means after rotation, 
# add this offset to get new position

{(0, 12): ((0, 1, 0), (1117, 115, 56)),
 (0, 13): ((1, 2, 1), (58, -22, 1216)),
 (0, 29): ((0, 3, 0), (-1298, -15, 88)),
 (1, 5): ((0, 0, 1), (-1261, 109, 120)),
 (1, 6): ((0, 1, 1), (-60, 1235, 24)),
 (2, 3): ((0, 3, 2), (-1084, 122, -167)),
 (2, 9): ((0, 0, 3), (149, 161, 1189)),
 (2, 30): ((1, 2, 2), (134, -1121, -142)),
 (3, 2): ((0, 3, 2), (-167, 122, -1084)),
 (3, 8): ((0, 0, 1), (-106, 1192, 45)),
 (3, 13): ((1, 0, 3), (-1358, -38, 66)),
 (3, 29): ((0, 1, 0), (-2, -31, 1194)),
 (4, 8): ((0, 2, 1), (1173, 78, -59)),
 (4, 13): ((1, 2, 3), (-79, 1308, -80)),
 (4, 23): ((0, 0, 1), (2, -1126, 15)),
 (5, 1): ((0, 0, 3), (-109, -1261, -120)),
 (5, 7): ((0, 1, 1), (-55, -54, 1183)),
 (5, 24): ((0, 3, 3), (-1187, 41, 18)),
 (5, 26): ((1, 2, 0), (1171, -29, -65)),
 (6, 1): ((1, 2, 1), (-24, -60, 1235)),
 (6, 8): ((1, 2, 2), (-1178, -7, -33)),
 (6, 16): ((0, 1, 0), (-31, -1310, -52)),
 (6, 23): ((1, 2, 0), (26, -81, -1204)),
 (6, 26): ((0, 1, 1), (31, 1172, -45)),
 (6, 28): ((0, 2, 3), (12

In [12]:
chains = get_chains(between_scanners.keys())
pprint(chains)
assert len(chains) == len(scans)

# this is how to convert one scanner back to scanner 0
# again, they should be reversed.

{0: None,
 1: [0, 29, 11, 26, 5, 1],
 2: [0, 13, 3, 2],
 3: [0, 13, 3],
 4: [0, 13, 4],
 5: [0, 29, 11, 26, 5],
 6: [0, 13, 3, 8, 6],
 7: [0, 29, 11, 26, 5, 7],
 8: [0, 13, 3, 8],
 9: [0, 13, 9],
 10: [0, 12, 10],
 11: [0, 29, 11],
 12: [0, 12],
 13: [0, 13],
 14: [0, 13, 9, 21, 14],
 15: [0, 29, 15],
 16: [0, 13, 3, 8, 30, 16],
 17: [0, 29, 15, 17],
 18: [0, 13, 3, 8, 30, 18],
 19: [0, 13, 9, 19],
 20: [0, 29, 11, 26, 5, 24, 20],
 21: [0, 13, 9, 21],
 22: [0, 29, 11, 26, 22],
 23: [0, 13, 4, 23],
 24: [0, 29, 11, 26, 5, 24],
 25: [0, 29, 11, 26, 5, 24, 20, 25],
 26: [0, 29, 11, 26],
 27: [0, 12, 27],
 28: [0, 13, 3, 8, 6, 28],
 29: [0, 29],
 30: [0, 13, 3, 8, 30]}


In [13]:
# debug. each transferred back then compare to scans[0]
for i in range(1,31):
    t = transfer_scanner_final(scans[i], chains[i], between_scanners)
    print(i, match_beacons(scans[0], t))
    
# NOTE: only the 3 scanner near scanner 0 share common points. 

1 []
2 []
3 []
4 []
5 []
6 []
7 []
8 []
9 []
10 []
11 []
12 [(((1, 0), (2, 21), (4, 22), (5, 19), (7, 20), (9, 13), (14, 1), (15, 24), (16, 25), (17, 4), (23, 9), (25, 5)), (0, 0, 0))]
13 [(((4, 16), (5, 4), (7, 18), (8, 19), (9, 20), (12, 21), (15, 2), (17, 13), (19, 3), (20, 25), (21, 1), (22, 15)), (0, 0, 0))]
14 []
15 []
16 []
17 []
18 []
19 []
20 []
21 []
22 []
23 []
24 []
25 []
26 []
27 []
28 []
29 [(((0, 4), (3, 2), (6, 8), (8, 18), (10, 22), (12, 13), (13, 16), (18, 6), (19, 15), (20, 25), (21, 9), (22, 17)), (0, 0, 0))]
30 []


# Q1

In [14]:
beacons = set([tuple((b for b in a)) for a in scans[0]])
# print(beacons)
for i in range(1, 31):
    t = transfer_scanner_final(scans[i], chains[i], between_scanners)
    # print(i, t)
    for t1 in t:
        beacons.add(t1)
    # print(i, len(beacons))

print(len(beacons))

372


# Q2

In [15]:
# find the positions of scanners related to scanner 0
scanners = [(0, 0, 0)]
for i in range(1, 31):
    t = transfer_scanner_final([(0,0,0)], chains[i], between_scanners)
    scanners.append(t[0])

pprint(scanners)

[(0, 0, 0),
 (-2462, 2362, 1290),
 (-1133, 138, 2366),
 (-1300, 16, 1282),
 (-21, 1286, 1296),
 (-2353, 2482, 29),
 (-1227, 2386, 1230),
 (-2408, 3665, 83),
 (-1194, 1208, 1237),
 (56, -23, 2515),
 (1248, 1236, -54),
 (-1298, 1269, -20),
 (1117, 115, 56),
 (58, -22, 1216),
 (1134, 1210, 2445),
 (-1297, 6, -1125),
 (-1175, 2355, 2540),
 (-2519, 76, -1207),
 (-2369, 1291, 2507),
 (-118, -32, 3562),
 (-4778, 2378, 95),
 (1240, 51, 2446),
 (-1175, 3679, -5),
 (-23, 2412, 1311),
 (-3540, 2500, -12),
 (-4857, 2520, -1229),
 (-1182, 2417, 58),
 (2423, 100, 112),
 (-1305, 3629, 1223),
 (-1298, -15, 88),
 (-1275, 1259, 2500)]


In [16]:
distances = [manhattan_distance(s1, s2) for s1 in scanners for s2 in scanners]
# print(distances)
print(max(distances))

12241
